Lệnh Docker bạn đã chạy:

```bash
docker run -d -p 9099:9099 --add-host=host.docker.internal:host-gateway -v pipelines:/app/pipelines --name pipelines --restart always ghcr.io/open-webui/pipelines:main
```

**Phân tích cụ thể:**
- **Image**: `ghcr.io/open-webui/pipelines:main`
  - Đây là **Docker image** được lưu trữ trên **GitHub Container Registry (GHCR)**, cụ thể là registry của tổ chức `open-webui`.
  - Image `pipelines:main` nằm trong repository `pipelines` của tổ chức `open-webui` trên GitHub.

**Câu lệnh này đang thực hiện những gì?**
1. **Tải Docker image từ GHCR**:
   - Docker sẽ tải image `ghcr.io/open-webui/pipelines:main` từ **GitHub Container Registry (GHCR)** nếu image chưa tồn tại trên máy của bạn.
   - `main` là tag của image. Đây thường là phiên bản mới nhất hoặc mặc định của dự án.

2. **Chạy container**:
   - Image `pipelines` được khởi chạy trên máy của bạn với các cấu hình sau:
     - **Port binding**: Cổng `9099` trên host (máy của bạn) được ánh xạ với cổng `9099` bên trong container.
     - **Volume**: Thư mục `pipelines` trên máy của bạn được gắn vào `/app/pipelines` bên trong container. Điều này cho phép dữ liệu được lưu trữ giữa các lần khởi động container.
     - **Host mapping**: Tên `host.docker.internal` được ánh xạ đến `host-gateway`, giúp container có thể giao tiếp với host.

3. **Cấu hình tự động khởi động lại**:
   - `--restart always`: Container sẽ tự động khởi động lại nếu bị dừng.

---

### **GitHub Container Registry (GHCR) là gì?**
- GHCR là một dịch vụ của GitHub cho phép lưu trữ Docker images.
- Image `ghcr.io/open-webui/pipelines` là một phần của dự án mã nguồn mở trên GitHub.

---

### **Kiểm tra image và thông tin thêm**
1. **Xác minh image đã được tải**:
   - Kiểm tra image trên máy của bạn:
     ```bash
     docker images
     ```
     Bạn sẽ thấy `ghcr.io/open-webui/pipelines` xuất hiện trong danh sách.

2. **Thông tin image trên GHCR**:
   - Truy cập [GitHub Container Registry của open-webui](https://github.com/open-webui/pipelines/pkgs/container/pipelines) để xem thêm chi tiết về image này.

---

Nếu bạn cần hướng dẫn tùy chỉnh lệnh hoặc tải image từ nguồn khác, hãy cho mình biết nhé! 😊

## Check UI ko có pipeline + Check Postman + Check Docker 
```
curl --location 'http://localhost:9099/api/tags'
```

### **Phân tích lỗi DOCKER**

Dựa trên log bạn cung cấp, vấn đề chính xuất phát từ lỗi **404 Not Found** khi truy cập endpoint `/api/tags`. Điều này chỉ ra rằng:

1. **Endpoint `/api/tags` không tồn tại** trên server Pipelines.
2. **Module hoặc pipeline chưa được định nghĩa hoặc kích hoạt** trên server Pipelines.
3. Server Pipelines đã khởi động nhưng không tải được các pipeline cụ thể cần thiết.

---

### **Nguyên nhân chính**

1. **Biến môi trường chưa được cấu hình đúng**:
   - Không có pipeline URL nào được chỉ định thông qua biến `PIPELINES_URLS`.
   - Dòng log:
     ```
     PIPELINES_URLS not specified. Skipping pipelines download and installation.
     ```
     Chỉ ra rằng server không tải bất kỳ pipeline nào.

2. **Reset thư mục pipelines không được thực hiện**:
   - Dòng log:
     ```
     RESET_PIPELINES_DIR is not set to true. No action taken.
     ```
     Nghĩa là server sử dụng cấu hình và pipelines cũ (nếu có). Nếu không có pipeline nào được lưu trước đó, server sẽ khởi động mà không có pipeline.

3. **Module haystack_pipeline đã được tải, nhưng không có pipeline được định nghĩa**:
   - Dòng log:
     ```
     Loaded module: haystack_pipeline
     ```
     Chỉ báo rằng module đã được tải, nhưng không liên kết với bất kỳ pipeline nào cung cấp endpoint `/api/tags`.

---

### **Hướng dẫn xử lý**

#### **1. Khởi động lại với cấu hình đúng**
Bạn cần cung cấp URL pipeline hoặc reset thư mục pipelines khi khởi động container. Thực hiện một trong các bước sau:

##### **Cách 1: Cung cấp `PIPELINES_URLS`**
Thêm pipeline mẫu từ GitHub:
```bash
docker run -d \
  -p 9099:9099 \
  --add-host=host.docker.internal:host-gateway \
  -e PIPELINES_URLS="https://github.com/open-webui/pipelines/blob/main/examples/filters/detoxify_filter_pipeline.py" \
  -v pipelines:/app/pipelines \
  --name pipelines \
  --restart always \
  ghcr.io/open-webui/pipelines:main
```

##### **Cách 2: Reset thư mục pipelines**
Nếu bạn muốn server tự khởi động lại với các pipeline mặc định:
```bash
docker run -d \
  -p 9099:9099 \
  --add-host=host.docker.internal:host-gateway \
  -e RESET_PIPELINES_DIR=true \
  -v pipelines:/app/pipelines \
  --name pipelines \
  --restart always \
  ghcr.io/open-webui/pipelines:main
```

---

#### **2. Xác minh API `/api/tags`**

Sau khi khởi động lại container, kiểm tra lại API:
```bash
curl --location 'http://localhost:9099/api/tags'
```
Nếu trả về danh sách pipeline, server đã hoạt động đúng.

---

#### **3. Cài đặt pipeline thủ công (nếu cần)**
Nếu bạn muốn thêm pipeline cụ thể:
1. **Tải pipeline từ GitHub**:
   ```bash
   wget -P ./pipelines https://raw.githubusercontent.com/open-webui/pipelines/main/examples/filters/detoxify_filter_pipeline.py
   ```

2. **Khởi động lại Pipelines server**:
   Nếu bạn không sử dụng Docker:
   ```bash
   sh ./start.sh
   ```

---

#### **4. Kiểm tra logs**
Kiểm tra lại logs sau khi thực hiện các bước trên:
```bash
docker logs pipelines
```
Xác minh rằng pipelines đã được tải thành công.

---

### **Nếu lỗi vẫn còn**
1. **Đọc thêm tài liệu chính thức**:
   - [GitHub Pipelines Documentation](https://github.com/open-webui/pipelines).

2. **Tham gia cộng đồng hỗ trợ**:
   - Tham gia [GitHub Discussions](https://github.com/open-webui/pipelines/discussions) hoặc Discord của dự án để nhận hỗ trợ từ cộng đồng.

Nếu bạn cần hỗ trợ thêm, hãy cung cấp thêm thông tin logs hoặc kết quả sau khi thực hiện các bước trên. Mình sẵn sàng hỗ trợ! 😊


```
docker run -d -p 9099:9099 --add-host=host.docker.internal:host-gateway -v pipelines:/app/pipelines -e PIPELINES_URLS="https://raw.githubusercontent.com/open-webui/pipelines/main/examples/pipelines/rag/haystack_pipeline.py" --name pipelines --restart always ghcr.io/open-webui/pipelines:main
```

# Bug : add nhầm đường link github - bị sai 



1. Add API Key vào openAI => Models tự động được tạo 
2. Với Ollama, 
- thì có sẵn ollama 2 sau khi kích hoạt hoặc chạy ollama dưới nền thì phải 
- Lúc đó đang chạy docker nên là bạn click vào : http://host.docker.internal:11434 để add thêm model từ huggingface nhé 

---
Dưới đây là lệnh **1 dòng** để chạy mỗi container trong terminal:

### **Chạy Pipelines**
```bash
docker run -d -p 9099:9099 --add-host=host.docker.internal:host-gateway -v pipelines:/app/pipelines -e PIPELINES_URLS="https://raw.githubusercontent.com/open-webui/pipelines/main/examples/pipelines/rag/haystack_pipeline.py" --name pipelines --restart always ghcr.io/open-webui/pipelines:main
```

---

### **Chạy Open WebUI**
```bash
docker run -d -p 3000:8080 -v open-webui:/app/backend/data --name open-webui --restart always ghcr.io/open-webui/open-webui:main
```

Cách set up này đủ để chạy openAI, ollama, còn pipeline thì đang bug. 

---

3. Với việc add pipeline thì: 
```
docker run -d -p 9099:9099 --add-host=host.docker.internal:host-gateway -v pipelines:/app/pipelines -e PIPELINES_URLS="https://raw.githubusercontent.com/open-webui/pipelines/blob/main/examples/pipelines/rag/haystack_pipeline.py" --name pipelines --restart always ghcr.io/open-webui/pipelines:main
```


Add : 
```
- Set the API URL to http://localhost:9099 and the API key to 0p3n-w3bu!.
- **Docker** : If your Open WebUI is running in a Docker container, replace localhost with `http://host.docker.internal:9099` in the API URL.
```

crul - url DOCKER: 
```
curl --location 'http://host.docker.internal:9099/api/tags'
```

Run container: 
```
docker run -d -p 9099:9099 --add-host=host.docker.internal:host-gateway -v pipelines:/app/pipelines --name pipelines --restart always ghcr.io/open-webui/pipelines:main
```

Sử dụng pipeline example có sẵn: 
```
docker run -d -p 9099:9099 --add-host=host.docker.internal:host-gateway -v pipelines:/app/pipelines -e PIPELINES_URLS="https://raw.githubusercontent.com/open-webui/pipelines/blob/main/examples/pipelines/rag/haystack_pipeline.py" --name pipelines --restart always ghcr.io/open-webui/pipelines:main
```

- Custom pipeline: 
```
docker run -d -p 9099:9099 --add-host=host.docker.internal:host-gateway -e PIPELINES_URLS="URL_TÙY_CHỈNH" -v pipelines:/app/pipelines --name pipelines --restart always ghcr.io/open-webui/pipelines:main
```

Installation and Setup
Get started with Pipelines in a few easy steps:

Ensure Python 3.11 is installed. It is the only officially supported Python version.

Clone the Pipelines repository:

git clone https://github.com/open-webui/pipelines.git
cd pipelines

Install the required dependencies:

pip install -r requirements.txt

Start the Pipelines server:

sh ./start.sh

# Sau khi click để kích hoạt : Pipeline 
- ./start.sh 
Tự động check 
```
INFO:     Uvicorn running on http://0.0.0.0:9099 (Press CTRL+C to quit)
INFO:     127.0.0.1:37212 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:59288 - "GET /api/tags HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:35926 - "GET /api/tags HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:36654 - "GET /api/tags HTTP/1.1" 404 Not Found

```
- hoặc docker pipeline works. 